In [9]:
from pathlib import Path
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.keras.layers import Dense
from numpy.random import default_rng
from sklearn.model_selection import train_test_split
import math
from squaternion import Quaternion


In [10]:
stop= pd.read_csv('data/stop_90_diff_quat.csv', sep=",")
m_0 = pd.read_csv('data/0_90_diff_quat.csv', sep=",")
m_90 = pd.read_csv('data/90_90_diff_quat.csv', sep=",")
m_180 = pd.read_csv('data/180_90_diff_quat.csv', sep=",")
m_270 = pd.read_csv('data/270_90_diff_quat.csv', sep=",")
m_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_y,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w
0,NaN,NaN,NaN,0.458186,0.466677,0.545427,0.524201,NaN,NaN,NaN,...,-0.258706,-0.458301,-0.629763,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000,0.000,0.000,0.458945,0.467419,0.544761,0.523569,0.000,0.000,0.000,...,-0.258706,-0.458301,-0.629763,0.000,-0.001,0.004,0.002,0.005,0.001,0.000
2,0.000,0.000,0.000,0.458945,0.467419,0.544761,0.523569,0.000,0.000,0.000,...,-0.258706,-0.458301,-0.629763,-0.002,-0.001,0.004,0.003,-0.002,-0.001,0.000
3,0.000,0.000,0.000,0.458945,0.467419,0.544761,0.523569,0.000,0.000,0.000,...,-0.258706,-0.458301,-0.629763,0.000,-0.001,0.003,0.004,-0.001,0.000,0.000
4,0.000,0.000,0.000,0.458943,0.467419,0.544069,0.524290,0.000,0.000,0.000,...,-0.258150,-0.456736,-0.630483,-0.002,-0.001,0.004,0.002,-0.001,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,0.113,0.031,0.030,0.446189,0.495686,0.513271,0.540151,0.000,-0.002,0.001,...,-0.417196,-0.514753,-0.527471,0.011,0.003,-0.006,0.003,-0.001,-0.006,0.000
23994,0.133,0.024,0.005,0.482954,0.577703,0.472317,0.458183,-0.002,-0.002,0.000,...,-0.416494,-0.516832,-0.526704,-0.001,-0.001,0.001,-0.005,0.004,0.000,0.000
23995,0.061,0.006,0.005,0.472347,0.592602,0.457157,0.465531,-0.002,-0.002,0.001,...,-0.415801,-0.513190,-0.528668,-0.012,-0.003,0.006,0.005,0.006,0.006,0.000
23996,-0.056,-0.014,-0.010,0.504165,0.549392,0.492549,0.448756,-0.002,-0.001,0.001,...,-0.416493,-0.517548,-0.527425,-0.023,0.001,0.010,0.001,-0.001,0.011,0.001


In [11]:
swaper_270_90=m_90.loc[:,["e_r_x", "e_r_y","e_r_z","e_r_w","d_r_x", "d_r_y","d_r_z","d_r_w"]]
swaper_270_90

,e_r_x,e_r_y,e_r_z,e_r_w,d_r_x,d_r_y,d_r_z,d_r_w
0,0.466677,-0.458186,-0.524201,0.545427,-0.258706,0.571340,0.629763,-0.458301
1,0.467419,-0.458945,-0.523569,0.544761,-0.258706,0.571340,0.629763,-0.458301
2,0.467419,-0.458945,-0.523569,0.544761,-0.258706,0.571340,0.629763,-0.458301
3,0.467419,-0.458945,-0.523569,0.544761,-0.258706,0.571340,0.629763,-0.458301
4,0.467419,-0.458943,-0.524290,0.544069,-0.258150,0.572050,0.630483,-0.456736
...,...,...,...,...,...,...,...,...
23993,0.495686,-0.446189,-0.540151,0.513271,-0.417196,0.531744,0.527471,-0.514753
23994,0.577703,-0.482954,-0.458183,0.472317,-0.416494,0.531037,0.526704,-0.516832
23995,0.592602,-0.472347,-0.465531,0.457157,-0.415801,0.533157,0.528668,-0.513190
23996,0.549392,-0.504165,-0.448756,0.492549,-0.416493,0.529623,0.527425,-0.517548


In [12]:
swaper_90_270=m_270.loc[:,["e_r_x", "e_r_y","e_r_z","e_r_w","d_r_x", "d_r_y","d_r_z","d_r_w"]]
swaper_90_270

,e_r_x,e_r_y,e_r_z,e_r_w,d_r_x,d_r_y,d_r_z,d_r_w
0,0.458186,0.466677,0.545427,0.524201,-0.571340,-0.258706,-0.458301,-0.629763
1,0.458945,0.467419,0.544761,0.523569,-0.571340,-0.258706,-0.458301,-0.629763
2,0.458945,0.467419,0.544761,0.523569,-0.571340,-0.258706,-0.458301,-0.629763
3,0.458945,0.467419,0.544761,0.523569,-0.571340,-0.258706,-0.458301,-0.629763
4,0.458943,0.467419,0.544069,0.524290,-0.572050,-0.258150,-0.456736,-0.630483
...,...,...,...,...,...,...,...,...
23993,0.446189,0.495686,0.513271,0.540151,-0.531744,-0.417196,-0.514753,-0.527471
23994,0.482954,0.577703,0.472317,0.458183,-0.531037,-0.416494,-0.516832,-0.526704
23995,0.472347,0.592602,0.457157,0.465531,-0.533157,-0.415801,-0.513190,-0.528668
23996,0.504165,0.549392,0.492549,0.448756,-0.529623,-0.416493,-0.517548,-0.527425


In [13]:
m_90.loc[:,["e_r_x", "e_r_y","e_r_z","e_r_w","d_r_x", "d_r_y","d_r_z","d_r_w"]]=swaper_90_270

In [14]:
m_270.loc[:,["e_r_x", "e_r_y","e_r_z","e_r_w","d_r_x", "d_r_y","d_r_z","d_r_w"]]=swaper_270_90
m_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_y,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w
0,NaN,NaN,NaN,0.466677,-0.458186,-0.524201,0.545427,NaN,NaN,NaN,...,0.571340,0.629763,-0.458301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000,0.000,0.000,0.467419,-0.458945,-0.523569,0.544761,0.000,0.000,0.000,...,0.571340,0.629763,-0.458301,0.000,-0.001,0.004,0.002,0.005,0.001,0.000
2,0.000,0.000,0.000,0.467419,-0.458945,-0.523569,0.544761,0.000,0.000,0.000,...,0.571340,0.629763,-0.458301,-0.002,-0.001,0.004,0.003,-0.002,-0.001,0.000
3,0.000,0.000,0.000,0.467419,-0.458945,-0.523569,0.544761,0.000,0.000,0.000,...,0.571340,0.629763,-0.458301,0.000,-0.001,0.003,0.004,-0.001,0.000,0.000
4,0.000,0.000,0.000,0.467419,-0.458943,-0.524290,0.544069,0.000,0.000,0.000,...,0.572050,0.630483,-0.456736,-0.002,-0.001,0.004,0.002,-0.001,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,0.113,0.031,0.030,0.495686,-0.446189,-0.540151,0.513271,0.000,-0.002,0.001,...,0.531744,0.527471,-0.514753,0.011,0.003,-0.006,0.003,-0.001,-0.006,0.000
23994,0.133,0.024,0.005,0.577703,-0.482954,-0.458183,0.472317,-0.002,-0.002,0.000,...,0.531037,0.526704,-0.516832,-0.001,-0.001,0.001,-0.005,0.004,0.000,0.000
23995,0.061,0.006,0.005,0.592602,-0.472347,-0.465531,0.457157,-0.002,-0.002,0.001,...,0.533157,0.528668,-0.513190,-0.012,-0.003,0.006,0.005,0.006,0.006,0.000
23996,-0.056,-0.014,-0.010,0.549392,-0.504165,-0.448756,0.492549,-0.002,-0.001,0.001,...,0.529623,0.527425,-0.517548,-0.023,0.001,0.010,0.001,-0.001,0.011,0.001


In [15]:
stop.to_csv('stop.csv', index=False, header=True)
m_0.to_csv('m_0.csv', index=False, header=True)
m_90.to_csv('m_90.csv', index=False, header=True)
m_180.to_csv('m_180.csv', index=False, header=True)
m_270.to_csv('m_270.csv', index=False, header=True)

In [16]:
m_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_y,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w
0,NaN,NaN,NaN,0.466677,-0.458186,-0.524201,0.545427,NaN,NaN,NaN,...,0.571340,0.629763,-0.458301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000,0.000,0.000,0.467419,-0.458945,-0.523569,0.544761,0.000,0.000,0.000,...,0.571340,0.629763,-0.458301,0.000,-0.001,0.004,0.002,0.005,0.001,0.000
2,0.000,0.000,0.000,0.467419,-0.458945,-0.523569,0.544761,0.000,0.000,0.000,...,0.571340,0.629763,-0.458301,-0.002,-0.001,0.004,0.003,-0.002,-0.001,0.000
3,0.000,0.000,0.000,0.467419,-0.458945,-0.523569,0.544761,0.000,0.000,0.000,...,0.571340,0.629763,-0.458301,0.000,-0.001,0.003,0.004,-0.001,0.000,0.000
4,0.000,0.000,0.000,0.467419,-0.458943,-0.524290,0.544069,0.000,0.000,0.000,...,0.572050,0.630483,-0.456736,-0.002,-0.001,0.004,0.002,-0.001,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,0.113,0.031,0.030,0.495686,-0.446189,-0.540151,0.513271,0.000,-0.002,0.001,...,0.531744,0.527471,-0.514753,0.011,0.003,-0.006,0.003,-0.001,-0.006,0.000
23994,0.133,0.024,0.005,0.577703,-0.482954,-0.458183,0.472317,-0.002,-0.002,0.000,...,0.531037,0.526704,-0.516832,-0.001,-0.001,0.001,-0.005,0.004,0.000,0.000
23995,0.061,0.006,0.005,0.592602,-0.472347,-0.465531,0.457157,-0.002,-0.002,0.001,...,0.533157,0.528668,-0.513190,-0.012,-0.003,0.006,0.005,0.006,0.006,0.000
23996,-0.056,-0.014,-0.010,0.549392,-0.504165,-0.448756,0.492549,-0.002,-0.001,0.001,...,0.529623,0.527425,-0.517548,-0.023,0.001,0.010,0.001,-0.001,0.011,0.001


In [ ]:
m_90